In [ ]:
#run if using colab
#uninstall if problem during bert tokenization then reinstall below
!pip uninstall bert-for-tf2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install bert-for-tf2

     |████████████████████████████████| 51kB 5.6MB/s 
  Created wheel for bert-for-tf2: filename=bert_for_tf2-0.14.7-cp36-none-any.whl size=30537 sha256=6f4d87b6cf3a61548ffe4890d279bd76148f06fc730cb398615526b74317bacd
  Stored in directory: /root/.cache/pip/wheels/e1/f8/e2/b98f79a6b8cc898d8e4102b83acb8a098df7d27500a2bac912
  Created wheel for py-params: filename=py_params-0.9.7-cp36-none-any.whl size=7302 sha256=bc2c9461ca0f1ae2a7ec6c8f6131b51ca734bbda13d1614c4aa51c366f042d31
  Stored in directory: /root/.cache/pip/wheels/67/f5/19/b461849a50aefdf4bab47c4756596e82ee2118b8278e5a1980
  Created wheel for params-flow: filename=params_flow-0.8.2-cp36-none-any.whl size=19475 sha256=e8f5e8b1d9ee8fe30834a079cb6d7ca1d8059d6c5ee17df321ca781498b5d808
  Stored in directory: /root/.cache/pip/wheels/08/c8/7f/81c86b9ff2b86e2c477e3914175be03e679e596067dc630c06
Successfully built bert-for-tf2 py-params params-flow


In [ ]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 8.0MB/s 


In [ ]:
!pip install emoji --upgrade

     |████████████████████████████████| 51kB 4.8MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49715 sha256=585022aba7adf2b93008d2efb3a1a4a0eee7fe96f21e56471d90f98ccb6a6c31
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
#cd to folder which contain required external files, TEXT_MODEL.py and text_preprocessing.py
%cd /content/drive/MyDrive/Colab\ Notebooks/cs410/CourseProject 

/content/drive/MyDrive/Colab Notebooks/cs410/CourseProject


In [ ]:
import tensorflow as tf 
import tensorflow_hub as hub 
from tensorflow.keras import layers
import bert
import numpy as np 
import pandas as pd 
import json
import re
import random
import math
#from TEXT_MODEL import TEXT_MODEL
from TEXT_PREPROCESSING import preprocess_text

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:

# LOADING DATA
categorized_tweets = pd.read_json('./data/train.jsonl', lines = True)
categorized_tweets.isnull().values.any()
print(categorized_tweets)

# PREPROCESSING DATA
tweets = []
data = list(categorized_tweets["response"])
print(data[0])
for d in data:
    tweets.append(preprocess_text(d))

y = categorized_tweets["label"]
y = np.array(list(map(lambda x: 1 if x=="SARCASM" else 0, y)))



            label  ...                                            context
0         SARCASM  ...  [A minor child deserves privacy and should be ...
1         SARCASM  ...  [@USER @USER Why is he a loser ? He's just a P...
2         SARCASM  ...  [Donald J . Trump is guilty as charged . The e...
3         SARCASM  ...  [Jamie Raskin tanked Doug Collins . Collins lo...
4         SARCASM  ...  [Man ... y ’ all gone “ both sides ” the apoca...
...           ...  ...                                                ...
4995  NOT_SARCASM  ...  [@USER Apologies for the inconvenience you fac...
4996  NOT_SARCASM  ...  [@USER 🤔 idk tho , I think I ’ m #hungry . But...
4997  NOT_SARCASM  ...  [@USER @USER @USER Peace to you , and two coun...
4998  NOT_SARCASM  ...  [Bernie Sanders told Elizabeth Warren in priva...
4999  NOT_SARCASM  ...  [PDP PROTEST BRAINSTORMING SESSION Deji : We n...

[5000 rows x 3 columns]
@USER @USER @USER I don't get this .. obviously you do care or you would've moved right

In [ ]:
# TOKENIZING DATA

BertTokenizer = bert.bert_tokenization.FullTokenizer

bert_layer = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/3", trainable=False)
vocabulary_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
to_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = BertTokenizer(vocabulary_file, to_lower_case)

def tokenize_tweets(data):
    return tokenizer.convert_tokens_to_ids(tokenizer.tokenize(data))
  ##vectorized tweet
tokenized_tweets = [tokenize_tweets(tweet) for tweet in tweets]  

# tokenized example
print(tweets[9])
print(tokenizer.tokenize(tweets[9]))


@USER @USER @USER responds to facts by tossing out frantic insults , then accuses others of being " triggered by facts " 🤣 😂 🤣
['@', 'user', '@', 'user', '@', 'user', 'responds', 'to', 'facts', 'by', 'tossing', 'out', 'frantic', 'insults', ',', 'then', 'accuse', '##s', 'others', 'of', 'being', '"', 'triggered', 'by', 'facts', '"', '[UNK]', '[UNK]', '[UNK]']


In [ ]:
#find the longest tweet in order to pad shorter tweets w zeros
maxlength = 0
for alist in tokenized_tweets:
    if len(alist) > maxlength:
        maxlength = len(alist)
        
maxlength   

90

In [ ]:
#pad the input vector, tweets, so all observations have the same length
from keras.preprocessing.sequence import pad_sequences
tokenized_tweets_padded = pad_sequences(tokenized_tweets, maxlen=maxlength, padding = 'post')
tokenized_tweets_padded[0]

array([ 1030,  5310,  1030,  5310,  1030,  5310,  1045,  2123,  1005,
        1056,  2131,  2023,  1012,  1012,  5525,  2017,  2079,  2729,
        2030,  2017,  2052,  1005,  2310,  2333,  2157,  2247,  1012,
        1012,  2612,  2017,  2787,  2000,  2729,  1998, 18792,  2014,
        1012,  1012,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0],
      dtype=int32)

In [ ]:
type(tokenized_tweets_padded)

numpy.ndarray

In [ ]:
df_x = pd.DataFrame(tokenized_tweets_padded)
df_x.shape

(5000, 90)

In [ ]:
df_x.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89
0,1030,5310,1030,5310,1030,5310,1045,2123,1005,1056,2131,2023,1012,1012,5525,2017,2079,2729,2030,2017,2052,1005,2310,2333,2157,2247,1012,1012,2612,2017,2787,2000,2729,1998,18792,2014,1012,1012,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1030,5310,1030,5310,2667,2000,6186,2055,1012,3331,2055,2032,1998,2010,10873,1998,2027,3830,3209,1059,24475,2515,2008,2191,7861,1029,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1030,5310,1030,5310,1030,5310,2002,3084,2019,9577,2055,1997,2769,2013,1996,5691,1010,15313,999,1001,4553,14406,24138,27268,6633,9316,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1030,5310,1030,5310,5564,8398,2180,1005,1056,2130,2713,2010,2938,7644,1998,2010,24249,12655,2056,2002,2001,1996,12873,4355,3076,2027,1005,2310,2412,4036,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1030,5310,1030,5310,3492,2469,1996,3424,1011,5367,4306,3555,2008,1000,7072,2001,2006,1996,10428,1000,1999,7313,1010,2205,1012,2027,2245,5367,2001,1000,27246,1000,1012,1001,2175,2361,1001,2283,11253,4115,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(len(tokenizer.vocab))

30522


In [ ]:
#randomize and split into test and train datasets X,y
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df_x, y, test_size = 0.20, shuffle = True, random_state = 33)

In [ ]:
X_train.shape


(4000, 90)

In [ ]:
from keras.layers import LSTM

input_length = X_train.shape[1]
VOCAB_LENGTH = len(tokenizer.vocab)
EMB_DIM = 200

model = tf.keras.Sequential()
model.add(layers.Embedding(VOCAB_LENGTH, EMB_DIM, input_length=input_length))
model.add(LSTM(units=64, activation='tanh' ))

model.add(layers.Flatten())

#model.add(layers.Dense(128, activation='relu'))




#model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(64))
model.add(layers.Dropout(rate= 0.5))
model.add(layers.Activation('relu'))


#model.add(layers.Dense(28, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
print(model.summary())



Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 90, 200)           6104400   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                67840     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
dense (Dense)                (None, 64)                4160      
_________________________________________________________________
dropout (Dropout)            (None, 64)                0         
_________________________________________________________________
activation (Activation)      (None, 64)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 6

In [ ]:
#AdamW not working???
from tensorflow_addons.optimizers import AdamW
import tensorflow_addons as tfa
step = tf.Variable(0, trainable=False)
schedule = tf.optimizers.schedules.PiecewiseConstantDecay(
    [10000, 15000], [1e-0, 1e-1, 1e-2])
# lr and wd can be a function or a tensor
lr = 1e-1 * schedule(step)
wd = lambda: 1e-4 * schedule(step)

# ...

optimizer = tfa.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

In [ ]:
# compile network
from tensorflow.keras import optimizers

opt = optimizers.Adam(learning_rate=.0005, beta_1=0.9, beta_2=0.999, amsgrad=False)
model.compile(loss='binary_crossentropy', optimizer= opt, metrics=['accuracy'])
# fit network
model.fit(X_train, y_train, batch_size = 32, validation_data = (X_test, y_test), epochs=30, verbose=1)
# evaluate
loss, acc = model.evaluate(X_test, y_test, verbose=1)
print('Test Accuracy: %f' % (acc*100))

Epoch 1/30
125/125 [==============================] - 9s 71ms/step - loss: 0.6936 - accuracy: 0.4972 - val_loss: 0.6936 - val_accuracy: 0.4950
Epoch 2/30
125/125 [==============================] - 8s 66ms/step - loss: 0.6937 - accuracy: 0.5017 - val_loss: 0.6930 - val_accuracy: 0.5140
Epoch 3/30
125/125 [==============================] - 8s 67ms/step - loss: 0.6913 - accuracy: 0.5238 - val_loss: 0.6790 - val_accuracy: 0.6030
Epoch 4/30
125/125 [==============================] - 8s 66ms/step - loss: 0.6193 - accuracy: 0.6890 - val_loss: 0.5827 - val_accuracy: 0.7260
Epoch 5/30
125/125 [==============================] - 8s 66ms/step - loss: 0.5516 - accuracy: 0.7667 - val_loss: 0.5909 - val_accuracy: 0.7170
Epoch 6/30
125/125 [==============================] - 8s 66ms/step - loss: 0.4947 - accuracy: 0.8062 - val_loss: 0.6495 - val_accuracy: 0.7270
Epoch 7/30
125/125 [==============================] - 8s 67ms/step - loss: 0.4631 - accuracy: 0.8263 - val_loss: 0.5910 - val_accuracy: 0.7410

In [ ]:
from sklearn.metrics import precision_recall_fscore_support
y_prediction = model.predict(X_test)
y_pred = []
for i in y_prediction:
    if i >= 0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)    


p,r,f,n = precision_recall_fscore_support(y_test, y_pred, average='macro')
print('p = ',p, ', r = ', r, ', f = ', f)



p =  0.7811278749439426 , r =  0.7808680868086808 , f =  0.78090337838987


In [ ]:
#use if you want to save the model
#model.save("/content/drive/My Drive/Colab Notebooks/cs410/network02c")

In [ ]:
# Predict using model
uncat_tweets = pd.read_json('./data/test.jsonl', lines = True)
un_tweets = []
uncat_data = list(uncat_tweets["response"])

for d in uncat_data:
    un_tweets.append(preprocess_text(d))
tokenized_un_tweets = [tokenize_tweets(tweet) for tweet in un_tweets]
print(str(len(un_tweets)))

In [ ]:
#perform check of input lengths between test and train data
count = 0
for alist in tokenized_un_tweets:
    if len(alist) > count:
        count = len(alist)
if maxlength < count:
    print('error: input of test data input len greater than train data- need to fix')
else:
    print("ok to proceed")    

In [ ]:
tokenized_untweets_padded = pad_sequences(tokenized_un_tweets, maxlen=maxlength, padding = 'post')
tokenized_untweets_padded[0]

In [ ]:


predictions = model.predict(tokenized_untweets_padded)

with open('answer.txt', 'w') as f:
    c = 1
    s_c = 0
    ns_c = 0
    for p in predictions:
        if p >= .5:
            f.write("twitter_" + str(c) + "," + "SARCASM\n")
            c += 1
            s_c += 1
        else:
            f.write("twitter_" + str(c) + "," + "NOT_SARCASM\n")
            c += 1
            ns_c += 1
print("# sarcasm: " + str(s_c))
print("# not sarcasm: " + str(ns_c))


In [ ]:
!git commit -m "network_04a"
